

# Description of the data
The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 


`**books**`

Contains data on books:

- `book_id`
- `author_id`
- `title`
- `num_pages` — number of pages
- `publication_date`
- `publisher_id`

`**authors**`

Contains data on authors:

- `author_id`
- `author`

`**publishers**`

Contains data on publishers:

- `publisher_id`
- `publisher`

`**ratings**`

Contains data on user ratings:

- `rating_id`
- `book_id`
- `username` — the name of the user who rated the book
- `rating`

`**reviews**`

Contains data on customer reviews:

- `review_id`
- `book_id`
- `username` — the name of the user who reviewed the book
- `text` — the text of the review

# Conecting to data

In [1]:
# import libraries & connect to data
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# print 5 rows of every table table - quick look:
from IPython.display import display
tables = ['books','authors','ratings','reviews','publishers']

for table in tables :
  display(pd.io.sql.read_sql('''
                    SELECT * FROM {}
                    LIMIT 5
                        '''.format(table),
                    con = engine))
  print()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


# Tasks

- ## Find the number of books released after January 1, 2000.


In [3]:
# define query for distinct book realeased after 2000-01-01
query = '''

SELECT COUNT(DISTINCT(title,publication_date)) AS Books_Released
FROM books
WHERE publication_date > '2000-01-01'


'''

In [4]:
books_released = pd.io.sql.read_sql(query, con = engine)
books_released

,books_released
0,819


In [5]:
'number of released books after 2000-01-01 is: {}'. format(books_released.iloc[0][0])

'number of released books after 2000-01-01 is: 819'

- ## Find the number of user reviews and the average rating for each book.


In [6]:
query = ''' 

SELECT title AS book_name, COUNT(rating) AS number_reviews, AVG(rating) AS avg_rating
FROM books
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY title
ORDER BY number_reviews DESC
'''

In [7]:
# run query in data using
books_rating = pd.io.sql.read_sql(query, con = engine)
books_rating

,book_name,number_reviews,avg_rating
0,Twilight (Twilight #1),160,3.662500
1,The Hobbit or There and Back Again,88,4.125000
2,The Catcher in the Rye,86,3.825581
3,Angels & Demons (Robert Langdon #1),84,3.678571
4,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.414634
...,...,...,...
994,Open House,2,2.500000
995,Where Are the Children?,2,4.000000
996,Black Hole,2,4.500000
997,From a Buick 8,2,4.500000


- ## Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).


In [8]:
query = ''' 
SELECT publisher,COUNT(DISTINCT(title)) AS num_books
FROM books
INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50 
GROUP BY publisher
ORDER BY num_books DESC
LIMIT 1
'''

In [9]:
# run query in data using
books_published = pd.io.sql.read_sql(query, con = engine)
books_published

,publisher,num_books
0,Penguin Books,42


In [10]:
'The publisher that realeased highest number of books with more than 50 pages is: {}'.format(books_published.iloc[0][1])

'The publisher that realeased highest number of books with more than 50 pages is: 42'

- ##  Identify the author with the highest average book rating (look only at books with at least 50 ratings).


In [11]:
query = '''SELECT
    authors.author,
    AVG(ratings.rating) AS average_rating
FROM
    (SELECT
        ratings.book_id, 
        books.author_id
    FROM
        ratings
    LEFT JOIN books ON books.book_id = ratings.book_id
    GROUP BY
        ratings.book_id,
        books.author_id
    HAVING
        COUNT (ratings.rating_id) >= 50) AS subquery
INNER JOIN ratings ON ratings.book_id = subquery.book_id
INNER JOIN authors ON authors.author_id = subquery.author_id
GROUP BY
    authors.author_id
HAVING
    COUNT(ratings.rating_id) >= 50
ORDER BY
    average_rating DESC
LIMIT 1;
'''

In [12]:
# run query in data using
author_high = pd.io.sql.read_sql(query, con = engine)
author_high

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097


In [13]:
'{} is the author with the highest avgerage book rating of {}'.format(author_high.iloc[0][0],author_high.iloc[0][1])

'J.K. Rowling/Mary GrandPré is the author with the highest avgerage book rating of 4.287096774193548'

- ## Find the average number of text reviews among users who rated more than 50 books.



In [14]:
query = '''
SELECT
    AVG(Sub.cnt) AS average_reviews
FROM
    (SELECT
        COUNT(DISTINCT reviews.review_id) AS cnt
    FROM
        reviews
    INNER JOIN ratings ON reviews.username = ratings.username
    GROUP BY 
        reviews.username
    HAVING 
        COUNT(DISTINCT ratings.rating_id) > 50) AS Sub;

'''

In [15]:
# run query in data using
avg_text = pd.io.sql.read_sql(query, con = engine)
avg_text

,average_reviews
0,24.333333


In [16]:
'Average number of text reviews amoth users who rated more than 50 books is {} text reviews'.format(avg_text.iloc[0][0])

'Average number of text reviews amoth users who rated more than 50 books is 24.333333333333332 text reviews'